In [2]:
import pandas as pd
train_pos=pd.read_csv('./data/imdb/train/pos.csv')
train_neg=pd.read_csv('./data/imdb/train/neg.csv')
test_pos=pd.read_csv('./data/imdb/test/pos.csv')
test_neg=pd.read_csv('./data/imdb/test/neg.csv')
print('imported data.....')

def clear_text(text):
    return text.replace('"', '').replace("'", "").replace("<br />","").replace(",","").replace("(","").replace(")","").replace("-","").replace("/","").replace("!","")

# train_pos['text']=train_pos['text'].apply(clear_text)
# print(train_pos[train_pos.id==0]['text'])
# for word in train_pos[train_pos.id==0]['text']:
#     print(word,end=' ')


# li=train_pos[train_pos.id==0]['text'].apply(lambda x:x.split('.'))
# for word in li:
#     print(word,end=' ')
# # print(train_pos['t'])

train_pos_sent=train_pos['text'].apply(clear_text).tolist()
train_neg_sent=train_neg['text'].apply(clear_text).tolist()
test_pos_sent=test_pos['text'].apply(clear_text).tolist()
test_neg_sent=test_neg['text'].apply(clear_text).tolist()


corpus=train_pos_sent+train_neg_sent+test_pos_sent+test_neg_sent


imported data.....


In [4]:
words=[]
for review in corpus:
    review_words=[]
    review_sents=review.split(".")
    for sent in review_sents:
        review_words.extend(sent.split(" "))
    words.append(review_words)

In [7]:
import keras

Using TensorFlow backend.


In [11]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D ,LSTM
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils

In [49]:
def model(input_shape,hidden_size,dense_layer1,dense_layer2):
    X_input=Input(input_shape)
    conv=Conv2D(32, (5, 5), input_shape=input_shape, activation='relu')
    X=conv(X_input)
    conv_output=conv.output_shape
    print conv_output
    X=Reshape((),input_shape=conv_output)(X)
    X=LSTM(hidden_size,return_sequences=True)(X)
    X=Dense(dense_layer1, activation='relu')(X)
    X=Dense(dense_layer2, activation='relu')(X)
    model = Model(inputs = X_input, outputs = X, name='sent_classifier')
    return model

In [54]:
!pip install gensim

    100% |################################| 23.6MB 670kB/s ta 0:00:011
    100% |################################| 1.4MB 212kB/s ta 0:00:01
    100% |################################| 61kB 260kB/s ta 0:00:01
    100% |################################| 133kB 1.5MB/s ta 0:00:01
    100% |################################| 122kB 201kB/s ta 0:00:01
    100% |################################| 143kB 248kB/s ta 0:00:01
    100% |################################| 61kB 401kB/s ta 0:00:01
    100% |################################| 153kB 238kB/s ta 0:00:01
    100% |################################| 4.7MB 1.8MB/s ta 0:00:01
    100% |################################| 61kB 168kB/s ta 0:00:01
    100% |################################| 552kB 390kB/s ta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... done
  Stored in directory: /root

In [55]:
from gensim.models import Word2Vec
new_model = Word2Vec.load('model.bin')

In [59]:
new_model.most_similar('sd')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'decent', 0.7905937433242798),
 (u'great', 0.7879338264465332),
 (u'bad', 0.7295764684677124),
 (u'nice', 0.72676682472229),
 (u'good,', 0.7112225294113159),
 (u'fine', 0.7102065086364746),
 (u'cool', 0.7087936401367188),
 (u'funny', 0.6726918816566467),
 (u'solid', 0.6544207334518433),
 (u'poor', 0.6500015258789062)]